# PQC Migration Risk Model - Interactive Demo

This notebook provides an interactive exploration of the PQC Migration Risk Model framework.

## Overview

The framework models:
- Adversarial quantum computing capability growth
- Risk to cryptographic algorithms (RQR)
- Organizational security posture (CAS)
- Migration strategy effectiveness (TCI)


In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import ipywidgets as widgets

# Import framework modules
import config
from core_logic import calculate_adversarial_capability, calculate_rqr, calculate_cas
from scenarios import get_migration_coverage

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Framework loaded successfully!")

## Part 1: Understanding Adversarial Capability Growth

The framework models quantum computing capability as:

log10(A(t)) = log10(A0) + g*t + epsilon(t)

Where:
- A0: Initial capability
- g: Growth rate
- epsilon: Stochastic noise


In [ ]:
def plot_capability_growth(growth_rate=0.5, years=15, num_simulations=10):
    """Visualize adversarial capability growth."""
    np.random.seed(42)
    t_array = np.arange(1, years + 1)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for i in range(num_simulations):
        noise = np.random.normal(0, config.SIGMA_EPSILON, years)
        capability = calculate_adversarial_capability(t_array, growth_rate, noise)
        ax.plot(t_array, capability, alpha=0.3, color='blue')
    
    # Plot mean trajectory
    noise_mean = np.zeros(years)
    cap_mean = calculate_adversarial_capability(t_array, growth_rate, noise_mean)
    ax.plot(t_array, cap_mean, color='red', linewidth=2, label='Mean trajectory')
    
    ax.set_xlabel('Years')
    ax.set_ylabel('log10(Adversarial Capability)')
    ax.set_title(f'Adversarial Capability Growth (g={growth_rate})')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Interactive widget
widgets.interact(
    plot_capability_growth,
    growth_rate=widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.1, description='Growth Rate:'),
    years=widgets.IntSlider(value=15, min=5, max=30, step=1, description='Years:'),
    num_simulations=widgets.IntSlider(value=10, min=1, max=50, step=5, description='Simulations:')
);

## Part 2: Residual Quantum Risk (RQR)

RQR measures the probability that an adversary can break a cryptographic algorithm:

RQR(t) = 1 / (1 + exp(-alpha * delta))

Where delta = log10(A(t)) - log10(C_attack)


In [ ]:
def plot_rqr_comparison(growth_rate=0.5, alpha=2.0, years=15):
    """Compare RQR for different algorithms."""
    np.random.seed(42)
    t_array = np.arange(1, years + 1)
    
    # Generate capability
    noise = np.random.normal(0, config.SIGMA_EPSILON, years)
    capability = calculate_adversarial_capability(t_array, growth_rate, noise)
    
    # Calculate RQR for different algorithms
    original_alpha = config.ALPHA
    config.ALPHA = alpha
    
    rqr_rsa = calculate_rqr(capability, config.COST_RSA_2048)
    rqr_kyber = calculate_rqr(capability, config.COST_KYBER_512)
    
    config.ALPHA = original_alpha
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(t_array, rqr_rsa, marker='o', color='red', linewidth=2, label='RSA-2048')
    ax.plot(t_array, rqr_kyber, marker='s', color='blue', linewidth=2, label='Kyber-512')
    ax.axhline(0.5, linestyle='--', color='gray', alpha=0.5, label='50% threshold')
    
    ax.set_xlabel('Years')
    ax.set_ylabel('Break Probability')
    ax.set_title(f'Residual Quantum Risk Comparison (g={growth_rate}, α={alpha})')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1.05)
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\nYear 5 Risk Assessment:")
    print(f"  RSA-2048:  {rqr_rsa[4]:.6f}")
    print(f"  Kyber-512: {rqr_kyber[4]:.6f}")
    
    critical_rsa = np.where(rqr_rsa > 0.5)[0]
    if len(critical_rsa) > 0:
        print(f"\nRSA-2048 reaches 50% risk at year {critical_rsa[0] + 1}")
    else:
        print(f"\nRSA-2048 remains below 50% risk")

# Interactive widget
widgets.interact(
    plot_rqr_comparison,
    growth_rate=widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.1, description='Growth Rate:'),
    alpha=widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.5, description='Alpha (α):'),
    years=widgets.IntSlider(value=15, min=5, max=30, step=1, description='Years:')
);

## Part 3: Migration Scenarios and CAS

Compare different migration strategies and their impact on organizational security posture.


In [ ]:
def plot_migration_scenarios(growth_rate=0.5):
    """Compare different migration scenarios."""
    np.random.seed(42)
    years = 15
    t_array = np.arange(1, years + 1)
    
    # Calculate Kyber risk
    noise = np.random.normal(0, config.SIGMA_EPSILON, years)
    capability = calculate_adversarial_capability(t_array, growth_rate, noise)
    rqr_kyber = calculate_rqr(capability, config.COST_KYBER_512)
    
    # Get scenarios
    scenarios = ["Aggressive", "Conservative", "Late_Start"]
    colors = {"Aggressive": "green", "Conservative": "orange", "Late_Start": "red"}
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot coverage
    for scenario in scenarios:
        coverage = get_migration_coverage(scenario, t_array)
        ax1.plot(t_array, coverage, marker='o', color=colors[scenario], 
                linewidth=2, label=scenario)
    
    ax1.set_xlabel('Years')
    ax1.set_ylabel('Migration Coverage')
    ax1.set_title('Migration Coverage Trajectories')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1.05)
    
    # Plot CAS
    for scenario in scenarios:
        coverage = get_migration_coverage(scenario, t_array)
        cas = calculate_cas(rqr_kyber, coverage)
        tci = np.mean(cas)
        ax2.plot(t_array, cas, marker='s', color=colors[scenario],
                linewidth=2, label=f'{scenario} (TCI={tci:.2f})')
    
    ax2.axhline(config.CAS_THRESHOLD, linestyle='--', color='black', 
               alpha=0.5, label='Threshold')
    ax2.set_xlabel('Years')
    ax2.set_ylabel('Composite Assurance Score')
    ax2.set_title('CAS Evolution by Scenario')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 1.05)
    
    plt.tight_layout()
    plt.show()

# Interactive widget
widgets.interact(
    plot_migration_scenarios,
    growth_rate=widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.1, 
                                   description='Growth Rate:')
);

## Part 4: Full Simulation

Run a complete Monte Carlo simulation with customizable parameters.


In [ ]:
def run_full_simulation(n_iterations=100, years=15, seed=42):
    """Run complete Monte Carlo simulation."""
    np.random.seed(seed)
    
    print(f"Running simulation with {n_iterations} iterations...")
    
    t_array = np.arange(1, years + 1)
    rqr_rsa_all = np.zeros((n_iterations, years))
    rqr_kyber_all = np.zeros((n_iterations, years))
    
    g_rates = np.random.normal(config.MU_G, config.SIGMA_G, n_iterations)
    
    for i in range(n_iterations):
        noise = np.random.normal(0, config.SIGMA_EPSILON, years)
        capability = calculate_adversarial_capability(t_array, g_rates[i], noise)
        rqr_rsa_all[i, :] = calculate_rqr(capability, config.COST_RSA_2048)
        rqr_kyber_all[i, :] = calculate_rqr(capability, config.COST_KYBER_512)
    
    # Plotting
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # RSA
    rsa_mean = np.mean(rqr_rsa_all, axis=0)
    rsa_ci = 1.96 * np.std(rqr_rsa_all, axis=0) / np.sqrt(n_iterations)
    ax.plot(t_array, rsa_mean, color='red', linewidth=2, label='RSA-2048')
    ax.fill_between(t_array, rsa_mean - rsa_ci, rsa_mean + rsa_ci, 
                    color='red', alpha=0.2)
    
    # Kyber
    kyber_mean = np.mean(rqr_kyber_all, axis=0)
    kyber_ci = 1.96 * np.std(rqr_kyber_all, axis=0) / np.sqrt(n_iterations)
    ax.plot(t_array, kyber_mean, color='blue', linewidth=2, label='Kyber-512')
    ax.fill_between(t_array, kyber_mean - kyber_ci, color='blue', alpha=0.2)
    
    ax.axhline(0.5, linestyle='--', color='gray', alpha=0.5)
    ax.set_xlabel('Years')
    ax.set_ylabel('Break Probability')
    ax.set_title(f'Monte Carlo Simulation Results ({n_iterations} iterations)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1.05)
    plt.tight_layout()
    plt.show()
    
    print("\nSimulation complete!")
    print(f"Year 5 RSA-2048 risk: {rsa_mean[4]:.6f}")
    print(f"Year 5 Kyber-512 risk: {kyber_mean[4]:.6f}")

# Interactive widget
widgets.interact(
    run_full_simulation,
    n_iterations=widgets.IntSlider(value=100, min=10, max=1000, step=50, 
                                  description='Iterations:'),
    years=widgets.IntSlider(value=15, min=5, max=30, step=5, description='Years:'),
    seed=widgets.IntSlider(value=42, min=1, max=100, step=1, description='Random Seed:')
);

## Conclusion

This interactive notebook demonstrates the key features of the PQC Migration Risk Model:

1. **Adversarial Capability**: Stochastic modeling of quantum computing growth
2. **Risk Assessment**: Quantifying break probability for different algorithms
3. **Migration Strategies**: Comparing organizational transition approaches
4. **Full Simulation**: Complete Monte Carlo analysis with uncertainty quantification

For production use, run the main simulation script:
```bash
python main.py --n-iterations 5000 --seed 42
```

See the repository README for more details and examples.
